In [105]:
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import StandardScaler

In [2]:
test = pd.read_csv('raif_test.csv')
train = pd.read_csv('raif_train.csv')

D:\1Soft\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [237]:
train.columns

Index(['city', 'floor', 'id', 'lat', 'lng', 'osm_amenity_points_in_0.001',
       'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
       'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
       'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist',
       'osm_city_nearest_name', 'osm_city_nearest_population',
       'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
       'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075',
       'osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001',
       'osm_culture_points_in_0.005', 'osm_culture_points_in_0.0075',
       'osm_culture_points_in_0.01', 'osm_finance_points_in_0.001',
       'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075',
       'osm_finance_points_in

In [153]:
fig = px.histogram(train['city'])
fig.show()

In [154]:
fig = px.histogram(test['city'])
fig.show()

In [90]:
test = pd.read_csv('raif_test.csv')
train = pd.read_csv('raif_train.csv')

D:\1Soft\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [91]:
train = train.loc[train['price_type'] == 1]

In [92]:
tmp_set = set(train['realty_type'].values)
for i, element in enumerate(tmp_set):
    train['realty_type'].replace(element, f'type_{i}', inplace=True)
    test['realty_type'].replace(element, f'type_{i}', inplace=True)

In [93]:
test.head(5)

,city,floor,id,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,...,reform_mean_year_building_1000,reform_mean_year_building_500,region,lat,lng,total_square,street,date,realty_type,price_type
0,Курск,1.0,COL_289284,7,55,85,117,0,0,0,...,1966.471591,1966.740260,Курская область,51.709255,36.147908,156.148996,S6983,2020-09-06,type_1,1
1,Сургут,1.0,COL_289305,8,70,112,140,0,0,0,...,1988.259259,1989.068182,Ханты-Мансийский АО,61.233240,73.462509,190.737943,S29120,2020-09-06,type_2,1
2,Тюмень,-1.0,COL_289318,3,28,67,122,0,0,0,...,1985.880282,1991.458333,Тюменская область,57.143110,65.554573,457.118051,S23731,2020-09-06,type_0,1
3,Иркутск,1.0,COL_289354,5,76,139,231,0,0,0,...,1947.073276,1941.657895,Иркутская область,52.281380,104.282975,66.503622,S14207,2020-09-06,type_1,1
4,Курск,NaN,COL_289399,8,105,189,279,0,0,2,...,1948.764151,1946.689655,Курская область,51.729706,36.194019,23.864915,S20658,2020-09-06,type_0,1


# ВОТ ЗДЕСЬ НАЧИНЕТСЯ ПРОТОПТИП

Сделоть еще стэандарт скейлер.

Добавить средние метрики по регионам/типам недвижки

In [94]:
target = train['per_square_meter_price']
train['floor'].fillna('unknown', inplace=True)
train.fillna(0, inplace=True)
train.drop(['id', 'lat', 'lng', 'street', 'date', 'per_square_meter_price'], axis=1, inplace=True)

In [95]:
from catboost import CatBoostRegressor

In [96]:
model = CatBoostRegressor(iterations=1000,
                          learning_rate=0.001,
                          verbose=False,
                          loss_function='RMSE',
                          cat_features=['city', 'region', 'realty_type', 'floor', 'osm_city_nearest_name'])

In [97]:
test_id = test['id']
test.drop(['id', 'lat', 'lng', 'street', 'date'], axis=1, inplace=True)
test['floor'].fillna('unknown', inplace=True)
test.fillna(0, inplace=True)

In [100]:
grid = {'l2_leaf_reg': [1, 9, 20],
        'learning_rate': [0.001, 0.01, 0.1],
        'depth': [4, 6, 8]
       #thread_count=None Здесь можно менять количество ядер
       }

grid_search_result = model.grid_search(grid, 
                                       X=train, 
                                       y=target,
                                       plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))


bestTest = 84944.74316
bestIteration = 999

0:	loss: 84944.7431598	best: 84944.7431598 (0)	total: 16.5s	remaining: 7m 7s

bestTest = 80085.45036
bestIteration = 717

1:	loss: 80085.4503587	best: 80085.4503587 (1)	total: 37.6s	remaining: 7m 50s

bestTest = 80608.74458
bestIteration = 32

2:	loss: 80608.7445834	best: 80085.4503587 (1)	total: 1m 1s	remaining: 8m 15s

bestTest = 85685.03282
bestIteration = 999

3:	loss: 85685.0328250	best: 80085.4503587 (1)	total: 1m 17s	remaining: 7m 27s

bestTest = 80293.29087
bestIteration = 282

4:	loss: 80293.2908695	best: 80085.4503587 (1)	total: 1m 37s	remaining: 7m 7s

bestTest = 80361.53498
bestIteration = 24

5:	loss: 80361.5349780	best: 80085.4503587 (1)	total: 1m 57s	remaining: 6m 52s

bestTest = 86415.24501
bestIteration = 999

6:	loss: 86415.2450069	best: 80085.4503587 (1)	total: 2m 14s	remaining: 6m 24s

bestTest = 80231.45994
bestIteration = 323

7:	loss: 80231.4599417	best: 80085.4503587 (1)	total: 2m 37s	remaining: 6m 13s

bestTest = 802

In [101]:
model_grid = CatBoostRegressor(iterations=2000,
                          verbose=False,
                          loss_function='RMSE',
                          **grid_search_result['params'],
                          cat_features=['city', 'region', 'realty_type', 'floor', 'osm_city_nearest_name'])                          

In [102]:
model_grid.fit(train, target)

In [ ]:
feature_importance = model_grid.get_feature_importance()
top_5_features = pd.Series(feature_importance, index=test.columns).sort_values(ascending=False)
top_5_features = pd.DataFrame({'feature': top_5_features.index, 'importance': top_5_features.values})

fig = px.pie(top_5_features, values='importance', names='feature', title='Доля влияния признака на модель')
fig.show()

In [103]:
predictions = model_grid.predict(test)

In [104]:
predictions

array([39987.75865036, 46850.4567522 , 39196.39996581, ...,
       52781.53157072, 63041.34243462, 50297.93659976])

In [106]:
result = pd.DataFrame({'id': test_id, 'per_square_meter_price': predictions})

In [107]:
result.to_csv('predictions.csv')